適用資料筆數較少（不適用台北等大筆資料）

In [56]:
import pandas as pd
import numpy as np

In [45]:
data = pd.read_csv("./Data/Kaohsiung/g2.csv",delimiter=';')

In [46]:
print(data)

                 date_time            E             N          H  Angle  Axis  \
0      2024-01-18 04:30:00  224095.7991  2.551570e+06  1518.4140    0.0   0.0   
1      2024-01-18 04:40:00  224095.7980  2.551570e+06  1518.4135  137.0   1.6   
2      2024-01-18 10:40:00  224095.7973  2.551570e+06  1518.3954  151.0  18.7   
3      2024-01-18 10:50:00  224095.8001  2.551570e+06  1518.3838  277.0  31.1   
4      2024-01-18 11:10:00  224095.8011  2.551570e+06  1518.3934  288.0  21.5   
...                    ...          ...           ...        ...    ...   ...   
31179  2024-12-30 09:20:00  224095.7558  2.551570e+06  1518.5164    2.0  19.8   
31180  2024-12-30 09:30:00  224095.7558  2.551570e+06  1518.5165    7.0  20.1   
31181  2024-12-30 09:40:00  224095.7561  2.551570e+06  1518.5175    1.0  20.3   
31182  2024-12-30 09:50:00  224095.7536  2.551570e+06  1518.5217   50.0  22.4   
31183  2024-12-30 12:00:00  224095.7455  2.551570e+06  1518.5070  209.0   8.6   

       Plate  EMove  NMove 

In [47]:
data['date_time'] = pd.to_datetime(data['date_time'])
data['date_time'] = data['date_time'].astype(int) / (10 ** 11)

In [48]:
print(data['date_time'])

0        17055522.0
1        17055528.0
2        17055744.0
3        17055750.0
4        17055762.0
            ...    
31179    17355504.0
31180    17355510.0
31181    17355516.0
31182    17355522.0
31183    17355600.0
Name: date_time, Length: 31184, dtype: float64


In [49]:
lag_features = ['E', 'N', 'H', 'TotalMove']
lag_steps = 3

for feature in lag_features:
    for lag in range(1, lag_steps + 1):
        data[f'{feature}_lag{lag}'] = data[feature].shift(lag)

data = data.dropna()

In [50]:
target = ['E', 'N', 'H', 'TotalMove']
features = ["date_time"] + [f"{feature}_lag1" for feature in lag_features] + [f"{feature}_lag2" for feature in lag_features] + [f"{feature}_lag3" for feature in lag_features]

X = data[features]
y = data[target]

In [51]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgbrModel=xgb.XGBRegressor()
xgbrModel.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [53]:
y_pred = xgbrModel.predict(X_test)

In [54]:
print(y_pred)

[[2.2409581e+05 2.5515702e+06 1.5184016e+03 1.3608475e+01]
 [2.2409581e+05 2.5515702e+06 1.5183916e+03 2.3255177e+01]
 [2.2409581e+05 2.5515702e+06 1.5183701e+03 5.2574825e+01]
 ...
 [2.2409581e+05 2.5515702e+06 1.5184326e+03 5.5149799e+01]
 [2.2409580e+05 2.5515700e+06 1.5183647e+03 3.1822809e+02]
 [2.2409581e+05 2.5515702e+06 1.5183687e+03 5.0750416e+01]]


In [55]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

E_mae = mean_absolute_error(y_test['E'], y_pred[:,0])
E_rmse = np.sqrt(mean_squared_error(y_test['E'], y_pred[:,0]))

N_mae = mean_absolute_error(y_test['N'], y_pred[:,1])
N_rmse = np.sqrt(mean_squared_error(y_test['N'], y_pred[:,1]))

H_mae = mean_absolute_error(y_test['H'], y_pred[:,2])
H_rmse = np.sqrt(mean_squared_error(y_test['H'], y_pred[:,2]))

TotalM_mae = mean_absolute_error(y_test['TotalMove'], y_pred[:,3])
TotalM_rmse = np.sqrt(mean_squared_error(y_test['TotalMove'], y_pred[:,3]))

results = pd.DataFrame({
    "Variable": ["E","N","H","TotalMove"],
    "MAE": [E_mae,N_mae,H_mae,TotalM_mae],
    "RMSE": [E_rmse,N_rmse,H_rmse,TotalM_rmse]
})

print("results:")
print(results)

results:
    Variable       MAE       RMSE
0          E  0.016394   0.028212
1          N  0.215479   0.218477
2          H  0.005335   0.024796
3  TotalMove  5.174515  28.960320
